# Generate 2PCF of catalog

In [1]:
import numpy as np

from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams.update({'font.size': 14})

from pycorr import TwoPointCorrelationFunction, TwoPointEstimator, NaturalTwoPointEstimator, project_to_multipoles, project_to_wp, utils, setup_logging
from cosmoprimo import *

from densitysplit import catalog_data, density_split
from bin.density_split_mocks_functions import split_density, compute_densitySplit_CCF
from densitysplit.utils import *
from densitysplit.corr_func import *

# To activate logging
setup_logging()

## Data and output directories

In [2]:
data_dir = '/feynman/work/dphp/mp270220/data/'
output_dir = '/feynman/work/dphp/mp270220/outputs/'

## Get data

In [3]:
#catalog_name = 'AbacusSummit_1Gpc_z1.175'
catalog_name = 'AbacusSummit_2Gpc_z1.175'
#catalog_name = 'AbacusSummit_2Gpc_z0.800'

catalog = catalog_data.Data.load(data_dir+catalog_name+'.npy')
catalog.shift_boxcenter(-catalog.offset)

In [4]:
z = catalog.redshift

# Abacus summit base cosmology
cosmo_abacus = fiducial.AbacusSummitBase()
bg = cosmo_abacus.get_background()
f = bg.Omega_m(z)**0.55

## Apply RSD

In [5]:
hz = bg.hubble_function(catalog.redshift)

catalog.set_rsd(hz=hz)

## Compute 2PCF

In [6]:
edges = (np.linspace(0., 150., 51), np.linspace(-1, 1, 201))
los = 'x'

In [7]:
result = TwoPointCorrelationFunction('smu', edges,
                                     data_positions1=catalog.positions_rsd,
                                     boxsize=catalog.boxsize,
                                     engine='corrfunc', nthreads=128,
                                     los = los)

[000000.88]  05-16 15:46  TwoPointCorrelationFunction  INFO     Using estimator <class 'pycorr.twopoint_estimator.NaturalTwoPointEstimator'>.
[000000.88]  05-16 15:46  TwoPointCorrelationFunction  INFO     Computing two-point counts D1D2.
[000370.51]  05-16 15:52  TwoPointCorrelationFunction  INFO     Analytically computing two-point counts R1R2.


In [8]:
result.save(output_dir+catalog.name+'_2PCF_RSD')

[000370.54]  05-16 15:52  NaturalTwoPointEstimator     INFO     Saving /feynman/work/dphp/mp270220/outputs/AbacusSummit_2Gpc_z1.175_2PCF_RSD.
